In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import os 
sys.path.append(os.path.abspath('../'))
import torch
from time import time
from tqdm import tqdm
import drjit as dr
import mitsuba as mi
import numpy as np
import matplotlib.pyplot as plt 
from IPython.display import clear_output
import torch.nn as nn
import torch.nn.functional as F


from convolutions import *
from utils_fns import *
from utils_general import update_sigma_linear, run_scheduler_step, plt_errors, show_with_error
from optimizations import *
from utils_optim import run_optimization, run_grad_optimization, run_cg_optimization, run_bfgs_optimization
from utils_general import run_scheduler_step
from utils_mitsuba import get_mts_rendering, render_smooth
from read_scenes import create_scene_from_xml

if torch.cuda.is_available():
    device = 'cuda'
    print("is available")
    mi.set_variant('cuda_ad_rgb')

is available


# Setup f(x), g(x)

In [14]:
def layer_f(weights, x):
    '''
    Analytic, for linear 3d x -> 4d output
    weight matrix with shape (4,3)
    '''
    return weights @ x
    
def loss_f(input):
    '''
    Loss function, s.t. second derivative is non-zero
    '''
    return torch.sum(input**4)
    
def f(weights, x):
    return loss_f(layer_f(weights, x))

def ddloss_f(input):
    '''
    Second derivative of loss function wrt input
    '''
    return 12*input**2

def dlayer_f(weights, x):
    '''
    Derivative of layer_f wrt weights
    each column would be x with shape (4,3)
    '''
    return torch.stack([x for i in range(4)])
# def analytical_dfdx(x):
    

In [13]:
x = torch.tensor([0.5, 0.5, 0.5], device=device, requires_grad=True)
weights = torch.rand((4,3), device=device, requires_grad=True)

print(ddloss_f(layer_f(weights, x))*dlayer_f(weights, x))
hess_f = torch.func.hessian(f)
print(weights)
print(hess_f(weights, x))

tensor([[0.7658, 0.8513, 0.7657],
        [0.5590, 0.0492, 0.4930],
        [0.4588, 0.3949, 0.4158],
        [0.7591, 0.7903, 0.4464]], device='cuda:0', requires_grad=True)
tensor([[[[4.2579, 4.2579, 4.2579],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]],

         [[4.2579, 4.2579, 4.2579],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]],

         [[4.2579, 4.2579, 4.2579],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000],
          [0.9094, 0.9094, 0.9094],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000],
          [0.9094, 0.9094, 0.9094],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000],
          [0.9094, 0.9094, 0.9094],
          [0.0000, 0.0